In [1]:
import requests
import json
import csv
import requests
import os
import datetime
import dateutil.rrule
from datetime import date, timedelta
import dateutil.relativedelta as relativedelta
import dateutil.rrule as rrule
import calendar
import glob
from nltk.probability import FreqDist
import pandas as pd




In [2]:
param={"client_id":"8_C4NSGbL05eUQksKleGYw",
      "client_secret":"Wfgy3xIJtzaEfSJfd4lGz5UwEZo9cT3PWoZ5h2q72zDNLxm0Tq35PT78v2Lu7fGJ",
      'grant_type': 'client_credentials'}
r=requests.post("https://api.yelp.com/oauth2/token",param)
r.json()





{'access_token': '5YSTNCxuTZHDmKDEJ15lI_ZQNNs0LJ--b6FHlIpm2cdiZbxijUbA1_4LYy6CTuYvT67SGrz0It76zXR75y0VdpaVwKmSlMTsD-KE2XK71LpIT8ZHKe4sh6hbiCnnWHYx',
 'expires_in': 14990317,
 'token_type': 'Bearer'}

In [75]:
access_token="5YSTNCxuTZHDmKDEJ15lI_ZQNNs0LJ--b6FHlIpm2cdiZbxijUbA1_4LYy6CTuYvT67SGrz0It76zXR75y0VdpaVwKmSlMTsD-KE2XK71LpIT8ZHKe4sh6hbiCnnWHYx"
url = 'https://api.yelp.com/v3/businesses/search'


headers={'Authorization': 'bearer %s' % access_token}

params = {'location': 'New York',
           'term':'Restaurants',
#            'pricing_filter': '1, 2',
#              'sort_by': 'rating',
          'offset':0,
          'limit':50
         }


r1=requests.get(url=url,params=params,headers=headers)
r1.json()

{'businesses': [{'categories': [{'alias': 'seafood', 'title': 'Seafood'}],
   'coordinates': {'latitude': 40.7263139141197,
    'longitude': -73.9864901976935},
   'display_phone': '(917) 408-3395',
   'distance': 2412.085390294,
   'id': 'upstate-new-york-2',
   'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/qJNkVkHcGWsprlouwX3siw/o.jpg',
   'is_closed': False,
   'location': {'address1': '95 1st Ave',
    'address2': '',
    'address3': '',
    'city': 'New York',
    'country': 'US',
    'display_address': ['95 1st Ave', 'New York, NY 10003'],
    'state': 'NY',
    'zip_code': '10003'},
   'name': 'Upstate',
   'phone': '+19174083395',
   'price': '$$',
   'rating': 4.5,
   'review_count': 1338,
   'transactions': [],
   'url': 'https://www.yelp.com/biz/upstate-new-york-2?adjust_creative=8_C4NSGbL05eUQksKleGYw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=8_C4NSGbL05eUQksKleGYw'},
  {'categories': [{'alias': 'tradamerican',
     'title': 'American (Tr

In [34]:
def writejson(offset,r):    
    with open(os.getcwd()+"/raw data/NY-Restaurant1/"+str(offset)+'.json', 'w',encoding='utf-8',errors='ignore') as outfile:
        json.dump(r, outfile)

In [58]:
def collectdata(offestlist):
    for num in offestlist:      
        params = {'location': 'New York',
#                   'term':'Restaurants',
#            'pricing_filter': '1, 2',
#              'sort_by': 'rating',
          'offset':num,
         'limit':50
           }
        r=requests.get(url=url,params=params,headers=headers)
        writejson(num,r.json())
        
        
        
        
        
        

In [60]:
offestlist=[0,1,2,3,4,5]

collectdata(offestlist)

In [6]:
def readjson(path):                                               #read json
    with open(path,'r',encoding='utf-8',errors='ignore') as df:
        data=json.load(df)
        return data
    
    
    
    

In [68]:
def writecsv(x):
    with open(os.getcwd()+"/NY-restaurant.csv", 'a') as outcsv:      
        writer=csv.writer(outcsv,delimiter=',', quotechar='|',quoting=csv.QUOTE_MINIMAL, lineterminator='\n')   
        ree=x["businesses"]
        for item in ree:
            c1=item["id"].replace(","," ")
            c2=item["name"].replace(","," ")
            c3=item["display_phone"]
            c4=item["rating"]
            c5=item["review_count"]
            if len(item["transactions"])==0:
                c6=None
            else:            
                c6=item["transactions"][0]
            c7=item["coordinates"]["latitude"]
            c8=item["coordinates"]["longitude"]
            c9=item["location"]["city"].replace(","," ")
            if item["location"]["address1"]==None:
                c10=None
            else:
                c10=item["location"]["address1"].replace(","," ")
            if item["location"]["address2"]==None:
                c11=None
            else:
                c11=item["location"]["address2"].replace(","," ")
            if item["location"]["address3"]==None:
                c12=None
            else:
                c12=item["location"]["address3"].replace(","," ")    
            c13=item["location"]["zip_code"].replace(","," ")
            c14=item["distance"]
            if len(item["categories"])>2:
                c15=item["categories"][0]["title"].replace(","," ")
                c16=item["categories"][1]["title"].replace(","," ")
                c17=item["categories"][2]["title"].replace(","," ")
            if len(item["categories"])==2:
                c15=item["categories"][0]["title"].replace(","," ")
                c16=item["categories"][1]["title"].replace(","," ")
                c17=None
            if len(item["categories"])==1:
                c15=item["categories"][0]["title"].replace(","," ")
                c16=None
                c17=None  
            if len(item["categories"])==0:
                c15=None
                c16=None
                c17=None 
            if 'price' not in item:
                c18=None
            else:
                c18=item['price']
            writer.writerow([c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c11,c12,c13,c14,c15,c16,c17,c18])
            
            
            
            

In [69]:
def writehead():
    with open(os.getcwd()+"/NY-restaurant.csv", 'a') as outcsv:      
        writer=csv.writer(outcsv,delimiter=',', quotechar='|',quoting=csv.QUOTE_MINIMAL, lineterminator='\n')   
        writer.writerow(["id","name","phone","rating","reviewcount","transactions","latitude","longitude","city","address1","address2","address3","zip_code","distance","categories1","categories2","categories3","price"])
    
    
    

In [70]:
def outputcsv(pathlist):
    for path in pathlist:
        r=readjson(path)
        writecsv(r)
        
        
        
        

In [76]:
pathlist=glob.glob(os.getcwd()+"/raw data/NY-Restaurant/*.json")


In [77]:
if os.path.exists(os.getcwd()+"/NY-restaurant.csv"):
    os.remove(os.getcwd()+"/NY-restaurant.csv") 
writehead()    
outputcsv(pathlist)




In [78]:
df=pd.read_csv((os.getcwd()+'/NY-restaurant.csv'),low_memory=False)



In [79]:
df.head()



,id,name,phone,rating,reviewcount,transactions,latitude,longitude,city,address1,address2,address3,zip_code,distance,categories1,categories2,categories3,price
0,upstate-new-york-2,Upstate,(917) 408-3395,4.5,1338,NaN,40.726314,-73.986490,New York,95 1st Ave,NaN,NaN,10003,2412.085390,Seafood,NaN,NaN,$$
1,atrium-dumbo-brooklyn,Atrium Dumbo,(718) 858-1095,4.0,383,NaN,40.703399,-73.990616,Brooklyn,15 Main St,NaN,NaN,11201,373.043531,American (Traditional),NaN,NaN,$$
2,amélie-new-york,Amélie,(212) 533-2962,4.5,1557,NaN,40.732700,-73.997660,New York,22 W 8th St,NaN,NaN,10011,3035.998345,French,Wine Bars,NaN,$$
3,burger-and-lobster-new-york,Burger & Lobster,(646) 833-7532,4.5,3148,NaN,40.740160,-73.993372,New York,39 W 19th St,NaN,NaN,10011,3861.196120,Seafood,Burgers,NaN,$$
4,the-alcove-sunnyside,The Alcove,(347) 813-4159,5.0,232,NaN,40.745906,-73.915103,Sunnyside,41-11 49th St,NaN,NaN,11104,8045.690139,Gastropubs,Tapas Bars,Breakfast & Brunch,$$


In [80]:
df["categories1"].unique()






array(['Seafood', 'American (Traditional)', 'French', 'Gastropubs',
       'Cocktail Bars', 'Korean', 'Thai', 'Italian', 'American (New)',
       'Bars', 'Salad', 'Shanghainese', 'Moroccan', 'Ramen',
       'Breakfast & Brunch', 'Comfort Food', 'Burgers', 'Japanese',
       'Mexican', 'Southern', 'Cajun/Creole', 'Wine Bars', 'Vietnamese',
       'Modern European', 'Filipino', 'Poke', 'Asian Fusion',
       'Latin American', 'Mediterranean', 'Sandwiches', 'Hot Pot',
       'Lounges', 'Pizza', 'Chinese', 'Pubs', 'Caribbean', 'Beer Gardens',
       'Cuban', 'Coffee & Tea', 'Barbeque', 'Dim Sum', 'Brazilian',
       'Spanish', 'Sushi Bars', 'Austrian', 'Himalayan/Nepalese',
       'Argentine', 'Tapas/Small Plates', 'Food Court', 'Desserts',
       'German', 'Butcher', 'Venezuelan', 'Cantonese', 'Cafes',
       'Middle Eastern', 'Tapas Bars', 'New Mexican Cuisine', 'Lebanese',
       'Malaysian', 'Irish', 'Diners', 'Indonesian', 'Greek', 'Cinema',
       'Hawaiian', 'Peruvian', 'Egyptian', 

In [82]:
df.drop_duplicates()


,id,name,phone,rating,reviewcount,transactions,latitude,longitude,city,address1,address2,address3,zip_code,distance,categories1,categories2,categories3,price
0,upstate-new-york-2,Upstate,(917) 408-3395,4.5,1338,NaN,40.726314,-73.986490,New York,95 1st Ave,NaN,NaN,10003,2412.085390,Seafood,NaN,NaN,$$
1,atrium-dumbo-brooklyn,Atrium Dumbo,(718) 858-1095,4.0,383,NaN,40.703399,-73.990616,Brooklyn,15 Main St,NaN,NaN,11201,373.043531,American (Traditional),NaN,NaN,$$
2,amélie-new-york,Amélie,(212) 533-2962,4.5,1557,NaN,40.732700,-73.997660,New York,22 W 8th St,NaN,NaN,10011,3035.998345,French,Wine Bars,NaN,$$
3,burger-and-lobster-new-york,Burger & Lobster,(646) 833-7532,4.5,3148,NaN,40.740160,-73.993372,New York,39 W 19th St,NaN,NaN,10011,3861.196120,Seafood,Burgers,NaN,$$
4,the-alcove-sunnyside,The Alcove,(347) 813-4159,5.0,232,NaN,40.745906,-73.915103,Sunnyside,41-11 49th St,NaN,NaN,11104,8045.690139,Gastropubs,Tapas Bars,Breakfast & Brunch,$$
5,dutch-freds-new-york-2,Dutch Freds,(646) 918-6923,4.5,342,NaN,40.760818,-73.987846,New York,307 W 47th St,NaN,NaN,10036,6180.808995,Cocktail Bars,Tapas/Small Plates,Burgers,$$
6,thursday-kitchen-new-york,Thursday Kitchen,(646) 755-8088,4.5,314,NaN,40.727505,-73.983803,New York,424 E 9th St,NaN,NaN,10009,2613.855588,Korean,American (New),Tapas/Small Plates,$$
7,thai-villa-new-york-2,Thai Villa,(212) 802-9999,4.5,420,delivery,40.739020,-73.990650,New York,5 E 19th St,G Floor,NaN,10003,3744.838102,Thai,NaN,NaN,$$
8,almar-brooklyn-2,AlMar,(718) 855-5288,4.0,261,NaN,40.702563,-73.989008,Brooklyn,111 Front St,NaN,NaN,11201,534.810655,Italian,NaN,NaN,$$
9,jane-new-york-2,Jane,(212) 254-7000,4.0,2577,NaN,40.727382,-74.000205,New York,100 W Houston St,NaN,NaN,10012,2489.262738,American (Traditional),Modern European,NaN,$$


In [ ]:
df1=df[["rating","categories1","price"]]
df1.head()



In [ ]:
type(df1.price)

In [ ]:
df2=df1.copy()

In [ ]:
df2['price']=df2['price'].replace('$$',2)
df2['price']=df2['price'].replace('$',1)
df2['price']=df2['price'].replace('$$$',3)
df2['price']=df2['price'].replace('$$$$',4)
df2['price']=df2['price'].replace('$$$$$',5)

In [ ]:
df2.head()


In [ ]:
df2['Quantity']=1

In [ ]:
dd=df2.groupby(["categories1"],as_index=False).count()

dfcount=dd[["categories1","Quantity"]]
dfcount.head()

In [ ]:
df3=df2.groupby(["categories1"],as_index=False).mean()
df3.head()

In [ ]:
df_rpq=pd.merge(dfcount,df3,on='categories1')
df_rpq.head()

In [ ]:
ByRating=df3.sort_values(["rating"],ascending=[False])
ByRating.head()

In [ ]:
ByPrice=df3.sort_values(["price"],ascending=[False])
ByPrice.head()



